<a href="https://colab.research.google.com/github/Abhijeetkhade11/KaggleCompetitions/blob/main/titanic_prediction_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd



In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()


In [ ]:
train.info()
train.isnull().sum()


In [ ]:
for df in [train, test]:
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)


In [ ]:
for df in [train, test]:
    df["Title"] = df["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

    df["Title"] = df["Title"].replace(
        ["Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"],
        "Rare"
    )
    df["Title"] = df["Title"].replace({"Mlle":"Miss","Ms":"Miss","Mme":"Mrs"})


In [ ]:
for title in train["Title"].unique():
    median_age = train.loc[train["Title"] == title, "Age"].median()
    train.loc[(train["Age"].isna()) & (train["Title"] == title), "Age"] = median_age
    test.loc[(test["Age"].isna()) & (test["Title"] == title), "Age"] = median_age


In [ ]:
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)
test["Fare"].fillna(test["Fare"].median(), inplace=True)


In [ ]:
for df in [train, test]:
    df["Sex"] = df["Sex"].map({"male": 0, "female": 1})


In [ ]:
train = pd.get_dummies(train, columns=["Embarked", "Title"], drop_first=True)
test = pd.get_dummies(test, columns=["Embarked", "Title"], drop_first=True)


In [ ]:
drop_cols = ["Name", "Ticket", "Cabin"]
train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)


In [ ]:
X = train.drop("Survived", axis=1)
y = train["Survived"]

X_test = test[X.columns]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_split=10,
    random_state=42
)

model.fit(X, y)


In [ ]:
predictions = model.predict(X_test)


In [ ]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})

submission.to_csv("submission.csv", index=False)
submission.head()
